In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings; warnings.simplefilter('ignore')

### Dataset  - User and Movie Metadata

In [64]:
links_small = pd.read_csv('./links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

data = pd.read_csv("./idmb_user_movie_data.csv")
del data['timestamp']  
data.head(5)

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [66]:
movie = pd.read_csv("./idmb_movie_data.csv")
movie.head(5)

,Movie_id,Movie_title,Video_Release_date,IMDb URL
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [67]:
critics = {}
for index,row in data.iterrows():
    try:
        critics[row['user_id']][movie['Movie_title'][row['item_id']-1]] = row['rating']   
    except Exception:
        critics[row['user_id']] = {movie['Movie_title'][row['item_id']-1]:row['rating']}

print(critics)


{196: {'Kolya (1996)': 3, 'Mrs. Doubtfire (1993)': 4, "Muriel's Wedding (1994)": 4, 'Shall We Dance? (1996)': 3, 'Stand by Me (1986)': 5, 'Ace Ventura: Pet Detective (1994)': 5, 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': 4, 'Raising Arizona (1987)': 4, 'Being There (1979)': 5, 'Truth About Cats & Dogs, The (1996)': 4, 'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)': 2, 'Birdcage, The (1996)': 4, 'English Patient, The (1996)': 5, 'Home Alone (1990)': 3, 'American President, The (1995)': 5, 'Babe (1995)': 5, 'Harold and Maude (1971)': 4, 'Up in Smoke (1978)': 4, 'Four Weddings and a Funeral (1994)': 3, 'While You Were Sleeping (1995)': 3, 'Men in Black (1997)': 2, 'Kids in the Hall: Brain Candy (1996)': 4, 'Groundhog Day (1993)': 3, 'Boogie Nights (1997)': 3, "Marvin's Room (1996)": 3, 'Cold Comfort Farm (1995)': 3, 'Adventures of Priscilla, Queen of the Desert, The (1994)': 4, 'Secrets & Lies (1996)': 5, 'Van, The (1996)': 3, 'Waiting for Guffman (1996)': 4, 'N

In [68]:
md = pd.read_csv('./movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [69]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [70]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [71]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)


 #### Finding Similarity using **Euclidean distance**

The basis of many measures of similarity and dissimilarity is euclidean distance. Here two vectors are two persons and elements of the vectors are the ratings given by them on the common movies.

In [72]:
def sim_distance(prefs,user_1,user_2):  # Get the list of shared_items  
    si={}       #stores 1 if the movie is rated by both the users, else stores 0
    for item in prefs[user_1]:    
        if item in prefs[user_2]:       
            si[item]=1
    # if they have no ratings in common, return 0  
    if len(si)==0: 
        return 0
    # Add up the squares of all the differences  
    for item in prefs[user_1]:
        if item in prefs[user_2]:
            sum_of_squares=sum([pow(prefs[user_1][item]-prefs[user_2][item],2)])                      
    return(1/(1+sum_of_squares)) 


#### Finding Similarity using **Jaccard's Distance**


The Jaccard index, also known as  (A n B) / (A u B). The Jaccard coefficient measures similarity between finite sample sets, and is defined as the size of the intersection divided by the size of the union of the sample sets.


In [73]:
def sim_jaccard(prefs, p1, p2):
    numerator = 0
    difference = 0
    for movie in prefs[p1]:
        if movie in prefs[p2]:
            numerator = numerator + 1
        else:
            difference = difference + 1
    denominator = len(prefs[p1].keys()) + difference        
    return numerator/denominator 


#### Finding Similarity using **Pearson's Correlation** 


A slightly more sophisticated way to determine the similarity between people’s interests is to use a Pearson correlation coefficient. The correlation coefficient is a measure of how well two sets of data fit on a straight line.
it tends to give better results in situations where the data isn’t well normalized—for example, if critics’ movie rankings are routinely more harsh than average.


In [74]:
def sim_pearson(prefs,p1,p2):  # Get the list of mutually rated items  
    si={}      #stores 1 if the movie is rated by both the users, else stores 0
    for item in prefs[p1]:    
        if item in prefs[p2]: 
            si[item]=1
    # Find the number of elements  
    n=len(si)
    # if they are no ratings in common, return 0  
    if n==0: 
        return 0
    # Add up all the preferences  
    sum1=sum([prefs[p1][it] for it in si])  
    sum2=sum([prefs[p2][it] for it in si])
    # Sum up the squares  
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])  
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    # Sum up the products  
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    # Calculate Pearson score  
    num=pSum-(sum1*sum2/n)  
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))  
    if den==0: 
        return 0
    r=num/den
    return r

#### Finding the top matching users for a particular user


In [75]:
# Number of results and similarity function are optional params. 
def topMatches(prefs,person,n,similarity):  # n = No. of outputs needed
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    scores.sort()  
    scores.reverse()  
    return scores[0:n]


In [76]:
topMatches(critics,2,5,sim_pearson) # Top 5 matches for user with userid 2

[(1.0, 914), (1.0, 607), (1.0, 426), (1.0, 187), (1.0, 167)]

### Call

In [77]:
sim_distance(critics,451,266) #Similarity score(Euclidean) of users having userid 451 and userid 266

0.5

In [78]:
sim_jaccard(critics,451,266)  #Similarity score(Jacard) of users having userid 451 and userid 266

0.043478260869565216

In [17]:
sim_pearson(critics,451,266)  #Similarity score(Pearson) of users having userid 451 and userid 266

-0.46188021535170054

##### Chart

In [79]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [80]:
build_chart('Romance').head(10)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


#### I. Content Based Recommender
I am going to build an engine that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. Since we will be using movie metadata (or content) to build this engine, this also known as Content Based Filtering.

I build two Content Based Recommenders based on:

    1. Movie Overviews and Taglines
    2. Movie Cast, Crew, Keywords and Genre


In [81]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')

smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

#####  1. Movie  Overview and Tagline Based Recommender

In [82]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [83]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [84]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [85]:
def get_recommendations(title,cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

#### Get Recommendation from movie description and tagline

In [86]:
get_recommendations('The Dark Knight').head(5)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
Name: title, dtype: object

#####  2. Movie Cast, Crew, Keywords and Genre Recommender

In [87]:

credits = pd.read_csv('./credits.csv')
keywords = pd.read_csv('./keywords.csv')

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

md.shape

(45463, 25)

In [88]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

smd = md[md['id'].isin(links_small)]
smd.shape


(9219, 28)

In [89]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
# smd['genres'] = smd['genres'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [90]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


In [91]:
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [92]:

smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [103]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [104]:
s

0           jealousy
0                toy
0                boy
0         friendship
0            friends
            ...     
41391    destruction
41391          kaiju
41391          toyko
41669          music
41669    documentary
Name: keyword, Length: 64407, dtype: object

In [105]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [107]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [108]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)

smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [109]:
get_recommendations('The Dark Knight',cosine_sim).head(10)

8031      The Dark Knight Rises
6218              Batman Begins
6623               The Prestige
2085                  Following
4145                   Insomnia
3381                    Memento
8613               Interstellar
7648                  Inception
5943                   Thursday
8927    Kidnapping Mr. Heineken
Name: title, dtype: object

In [111]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [112]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


II. Collaborative Filtering

#### SVD (Singular Value Decomposition) 
For most recommendation algorithms, the input matrix being very sparse, matrix factorization methods are key since the space needs to be ‘reduced’ to a smaller latent one. SVD plays a central role in it.

In [141]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate ,train_test_split
from surprise import accuracy
from collections import defaultdict

reader = Reader()
ratings = pd.read_csv('./ratings_small.csv')
ratings.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [129]:

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

# Use the build_testset method to get the test set
testset = trainset.build_testset()
# data.split(n_folds=5)

In [132]:
svd = SVD()
svd.fit(trainset)
predictions = svd.test(testset)
predictions
# evaluate(svd, data, measures=['RMSE', 'MAE'])

[Prediction(uid=1, iid=31, r_ui=2.5, est=2.489962211240054, details={'was_impossible': False}),
 Prediction(uid=1, iid=1029, r_ui=3.0, est=2.8493793820613713, details={'was_impossible': False}),
 Prediction(uid=1, iid=1061, r_ui=3.0, est=2.628146601698489, details={'was_impossible': False}),
 Prediction(uid=1, iid=1129, r_ui=2.0, est=2.4713387620914085, details={'was_impossible': False}),
 Prediction(uid=1, iid=1172, r_ui=4.0, est=3.3774908246959567, details={'was_impossible': False}),
 Prediction(uid=1, iid=1263, r_ui=2.0, est=2.777989166354382, details={'was_impossible': False}),
 Prediction(uid=1, iid=1287, r_ui=2.0, est=2.8092175202633927, details={'was_impossible': False}),
 Prediction(uid=1, iid=1293, r_ui=2.0, est=2.8200814894207795, details={'was_impossible': False}),
 Prediction(uid=1, iid=1339, r_ui=3.5, est=2.7948270666973967, details={'was_impossible': False}),
 Prediction(uid=1, iid=1343, r_ui=2.0, est=2.8537751270255596, details={'was_impossible': False}),
 Prediction(uid

In [133]:
accuracy.rmse(predictions)

RMSE: 0.6442


0.6441581272321509

In [134]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [135]:
ratings[ratings['userId'] == 1]


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [144]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.5990799147323806, details={'was_impossible': False})

In [146]:
predictions

[Prediction(uid=1, iid=31, r_ui=2.5, est=2.489962211240054, details={'was_impossible': False}),
 Prediction(uid=1, iid=1029, r_ui=3.0, est=2.8493793820613713, details={'was_impossible': False}),
 Prediction(uid=1, iid=1061, r_ui=3.0, est=2.628146601698489, details={'was_impossible': False}),
 Prediction(uid=1, iid=1129, r_ui=2.0, est=2.4713387620914085, details={'was_impossible': False}),
 Prediction(uid=1, iid=1172, r_ui=4.0, est=3.3774908246959567, details={'was_impossible': False}),
 Prediction(uid=1, iid=1263, r_ui=2.0, est=2.777989166354382, details={'was_impossible': False}),
 Prediction(uid=1, iid=1287, r_ui=2.0, est=2.8092175202633927, details={'was_impossible': False}),
 Prediction(uid=1, iid=1293, r_ui=2.0, est=2.8200814894207795, details={'was_impossible': False}),
 Prediction(uid=1, iid=1339, r_ui=3.5, est=2.7948270666973967, details={'was_impossible': False}),
 Prediction(uid=1, iid=1343, r_ui=2.0, est=2.8537751270255596, details={'was_impossible': False}),
 Prediction(uid

##### Get Top Recommendation form SVD

In [159]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, r_ui, est, _ in predictions:
        top_n[uid].append((iid, est))
        for uid, user_ratings in top_n.items():
          user_ratings.sort(key=lambda x: x[1], reverse=True)
          top_n[uid] = user_ratings[:n]
    return top_n

In [160]:
top_n = get_top_n(predictions, n=10)
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])


1 [1172, 1953, 3671, 2105, 2150, 1343, 1029, 1293, 1287, 1339]
2 [527, 589, 110, 222, 590, 265, 551, 296, 497, 150]
3 [318, 356, 2318, 296, 2959, 2858, 3949, 1197, 110, 50068]
4 [296, 858, 913, 1258, 2064, 1214, 1198, 1197, 1079, 1219]
5 [1035, 356, 2762, 5995, 150, 30749, 1682, 903, 1961, 4995]
6 [2019, 5952, 1276, 7153, 111, 1250, 293, 1204, 1259, 903]
7 [260, 1148, 1223, 318, 745, 1225, 1196, 1198, 1210, 720]
8 [318, 4226, 858, 527, 2959, 2329, 33166, 1219, 593, 2762]
9 [527, 608, 318, 534, 1704, 1682, 2571, 36, 497, 1357]
10 [50, 2571, 1210, 1719, 318, 1704, 1196, 1197, 1611, 2344]
11 [296, 50, 48516, 1201, 923, 79132, 778, 77455, 26614, 96079]
12 [1387, 2959, 1220, 1235, 3879, 3408, 1215, 538, 1077, 3871]
13 [58559, 7502, 88125, 318, 527, 4993, 1259, 78499, 4878, 64614]
14 [1196, 3114, 3175, 3751, 2716, 2355, 2394, 2038, 2628, 1721]
15 [1193, 296, 111, 115569, 541, 3730, 904, 306, 149, 593]
16 [2858, 50, 1704, 318, 6016, 527, 4995, 1653, 750, 6711]
17 [1089, 1206, 858, 111, 541, 9

 The model recommended that the user associated with id = 1 should watch the movie associated with the id = 1172.

#### III. Hybrid Recommender

In [138]:
id_map = pd.read_csv('./links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [139]:
indices_map = id_map.set_index('id')

def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [140]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.357808
1011,The Terminator,4208.0,7.4,1984,218,3.211876
974,Aliens,3282.0,7.7,1986,679,3.088444
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.906124
9004,Suicide Squad,7717.0,5.9,2016,297761,2.832609
4278,Mothra vs. Godzilla,38.0,6.4,1964,1682,2.831016
344,True Lies,1138.0,6.8,1994,36955,2.807076
8445,Justice League: The Flashpoint Paradox,458.0,7.3,2013,183011,2.724531
8725,Teenage Mutant Ninja Turtles,2677.0,5.8,2014,98566,2.706250
922,The Abyss,822.0,7.1,1989,2756,2.668875
